In [25]:
from PIL import Image
from matplotlib import pyplot as plt

def sobrepor_imagens(imagem_a, imagem_b, posicao=(0, 0)):
    # Abre as imagens
    img_a = Image.open(imagem_a).convert("RGBA")
    img_b = Image.open(imagem_b).convert("RGBA")

    # Redimensiona a imagem A para ter as mesmas dimensões da imagem B
    img_a = img_a.resize(img_b.size, Image.Resampling.LANCZOS)

    # Cria uma nova imagem com o mesmo tamanho da imagem B
    nova_imagem = Image.new('RGBA', img_b.size)

    # Coloca a imagem A na nova imagem
    nova_imagem.paste(img_a, posicao, img_a)

    # Coloca a imagem B na nova imagem (pode comentar ou ajustar para outras necessidades)
    nova_imagem.paste(img_b, (0, 0), img_b)

    return nova_imagem

# Exemplo de uso
imagem_a = "as2.jpg"
imagem_b = "tenis segmentação.png"
nova_imagem = sobrepor_imagens(imagem_a, imagem_b)

# Salvar a nova imagem
nova_imagem.save("nova_imagem.png")

plt.show(nova_imagem)


In [21]:
import tkinter as tk
from tkinter import ttk, filedialog
from PIL import Image, ImageTk
import cv2 #Biblioteca de visão computacional 
import numpy as np
from rembg import remove ## biblioteca aom IA

class ImageProcessor:
    def __init__(self, root, canvas, output_label):
        self.root = root
        self.canvas = canvas
        self.output_label = output_label
        self.original_img = None
        self.processed_img = None
        self.background_img = None
        self.rect_start_x = None
        self.rect_start_y = None
        self.rect_end_x = None
        self.rect_end_y = None
        self.create_widgets()

    def create_widgets(self):
        self.canvas.grid(row=0, column=0, padx=20, pady=20)
        self.output_label.grid(row=0, column=1, padx=20, pady=20)

        button_browse_image = ttk.Button(self.root, text="Select Image", command=self.select_image)
        button_browse_image.grid(row=1, column=0, pady=(10, 10))

        button_remove_bg = ttk.Button(self.root, text="Remover com IA", command=self.remove_background)
        button_remove_bg.grid(row=1, column=2, pady=(10, 10))

        button_apply_grabcut = ttk.Button(self.root, text="Segmentação", command=self.apply_grabcut)
        button_apply_grabcut.grid(row=1, column=3, pady=(10, 10))

        button_save_removed_bg = ttk.Button(self.root, text="Salvar IA", command=self.save_removed_background)
        button_save_removed_bg.grid(row=2, column=0, pady=(10, 10))

        button_save_grabcut = ttk.Button(self.root, text="Salvar Segmentação", command=self.save_grabcut_result)
        button_save_grabcut.grid(row=2, column=3, pady=(10, 10))

        # Adiciona eventos de mouse para a seleção do retângulo
        self.canvas.bind("<ButtonPress-1>", self.on_rect_start)
        self.canvas.bind("<B1-Motion>", self.on_rect_drag)
        self.canvas.bind("<ButtonRelease-1>", self.on_rect_end)

    def select_image(self):
        # Limpa os dados existentes ao selecionar uma nova imagem
        self.reset()

        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
        if file_path:
            image = Image.open(file_path)
            self.original_img = image
            self.display_image(image, self.canvas)

    def reset(self): ## ao carregar imagem resetar todos os valores
        # Limpa os dados existentes
        self.original_img = None
        self.processed_img = None
        self.background_img = None
        self.rect_start_x = None
        self.rect_start_y = None
        self.rect_end_x = None
        self.rect_end_y = None
        self.canvas.delete("rect")  # Limpa o retângulo no canvas

    def remove_background(self): ## remove o fundo da imagem usando rembg
        if self.original_img:
            processed_img = remove(self.original_img)
            self.processed_img = processed_img
            self.display_output_image(processed_img)

    def apply_grabcut(self):
        if self.original_img:
            print("Applying GrabCut")
            img_cv2 = np.array(self.original_img)
            img_cv2 = cv2.cvtColor(np.array(self.original_img), cv2.COLOR_RGBA2BGR)  

            mask = np.zeros(img_cv2.shape[:2], np.uint8)
            rect = (
                int(self.rect_start_x),
                int(self.rect_start_y),
                int(self.rect_end_x - self.rect_start_x),
                int(self.rect_end_y - self.rect_start_y),
            )

            modeloFundo = np.zeros((1, 65), np.float64)
            modeloObjeto = np.zeros((1, 65), np.float64)
            
            # Invoca o GrabCut
            #o 5 é o numero de iterações quanto maior as iterações mais chances tem de melhorar
            #o GC_INIT_WITH_RECT é o modo de inicialização do grabcut, nesse caso é com um retangulo
            cv2.grabCut(img_cv2, mask, rect, modeloFundo, modeloObjeto, 5, cv2.GC_INIT_WITH_RECT)
            
            #valor 0 -> posição é fundo
            #valor 1 -> pregião faz parte do objeto final
            #valor 2 -> região é provavelmente fundo
            #valor 3 -> região é provavelmente objeto
            
            # cria uma mascara para a imagem
            # np.where -> troca os valores -> se a posição da máscara for 2 ou 0, então o valor é 0, se não é 1 
            # converte pra inteiro de 8 bits
            mask_final = np.where((mask == 2) | (mask == 0), 0, 1).astype("uint8")
            img_final = img_cv2 * mask_final[:, :, np.newaxis]

            # Aplica a máscara à imagem original
            for x in range(0, img_cv2.shape[0]):
                for y in range(0, img_cv2.shape[1]):
                    if mask_final[x, y] == 0:
                        img_final[x][y][0] = img_final[x][y][1] = img_final[x][y][2] = 255

            img_final = cv2.cvtColor(img_final, cv2.COLOR_BGR2RGB)
            self.img_final_pil = Image.fromarray(img_final)
            
            #remove o fundo da imagem preenchido por pixels brancos
            self.img_final_pil = remove(self.img_final_pil)
            
            #converte a imagem pill para png e salva
            self.img_final_pil.save('grabcut.png')

            self.display_output_image(self.img_final_pil)

    def save_removed_background(self): # salva a imagem sem fundo pela IA
        if self.processed_img:
            file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png")])
            if file_path:
                self.processed_img.save(file_path)

    def save_grabcut_result(self): # salva a imagem sem fundo pela segmentação
        if self.original_img:
            #converte a imagem pill para png e salva
            file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png")])
            if file_path:
                self.img_final_pil.save(file_path)

    def display_image(self, img, target): # exibe a imagem no canvas
        img.thumbnail((300, 300))
        img_tk = ImageTk.PhotoImage(img)
        target.configure(width=300, height=300)
        target.create_image(0, 0, anchor="nw", image=img_tk)
        target.image = img_tk

    def display_output_image(self, img): # exibe a imagem no label
        img.thumbnail((300, 300))
        img_tk = ImageTk.PhotoImage(img)
        self.output_label.configure(image=img_tk)
        self.output_label.image = img_tk

    def on_rect_start(self, event): # evento de click do mouse
        self.rect_start_x = event.x
        self.rect_start_y = event.y

    def on_rect_drag(self, event): # evento de arrastar o mouse
        self.rect_end_x = event.x
        self.rect_end_y = event.y
        self.canvas.delete("rect")
        self.canvas.create_rectangle(
            self.rect_start_x, self.rect_start_y, self.rect_end_x, self.rect_end_y, outline="blue", tags="rect"
        )

    def on_rect_end(self, event): # evento de soltar o mouse
        pass

def main():
    window = tk.Tk() # cria a janela
    window.title("Image Processor") # titulo da janela

    style = ttk.Style()
    style.configure("TButton", padding=(10, 5), font=("Helvetica", 12), background="#4CAF50", foreground="black") #estilo do botão
    style.configure("TLabel", font=("Helvetica", 12), background="#f0f0f0", padding=(10, 5), anchor="w") #estilo do label
    style.configure("TEntry", font=("Helvetica", 12), padding=(10, 5)) #estilo da entrada de texto
    style.configure("TFrame", background="#f0f0f0") #estilo do frame
    style.configure("TCheckbutton", background="#f0f0f0") #estilo do checkbutton

    frame = ttk.Frame(window) #cria o frame
    frame.grid(row=0, column=0, padx=20, pady=20) #posiciona o frame

    canvas = tk.Canvas(frame, relief="solid", borderwidth=1, width=400, height=300) #cria o canvas
    output_label = ttk.Label(frame, style="TLabel", relief="solid", borderwidth=1) #cria o label

    image_processor = ImageProcessor(window, canvas, output_label) #cria o objeto da classe ImageProcessor

    window.mainloop() #loop da janela

if __name__ == "__main__":
    main()


Applying GrabCut


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\yagok\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\yagok\AppData\Local\Temp\ipykernel_28796\2816161926.py", line 81, in apply_grabcut
    int(self.rect_start_x),
TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'


Applying GrabCut


In [31]:
from tkinter import Tk, Canvas, PhotoImage, Button, filedialog
from PIL import Image, ImageTk

class SobreposicaoImagensApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Sobreposição de Imagens")

        # Variáveis para armazenar as imagens
        self.imagem_a = None
        self.imagem_b = None

        # Variáveis para armazenar as posições x e y
        self.posicao_x = 0
        self.posicao_y = 0

        # Canvas para exibir as imagens
        self.canvas = Canvas(root)
        self.canvas.pack()

        # Botões para carregar imagens e realizar a sobreposição
        self.botao_imagem_a = Button(root, text="Carregar Imagem A", command=self.carregar_imagem_a)
        self.botao_imagem_a.pack()

        self.botao_imagem_b = Button(root, text="Carregar Imagem B", command=self.carregar_imagem_b)
        self.botao_imagem_b.pack()

        self.botao_sobrepor = Button(root, text="Sobrepor Imagens", command=self.sobrepor_imagens)
        self.botao_sobrepor.pack()

    def carregar_imagem_a(self):
        # Abrir o diálogo de seleção de arquivo
        caminho_imagem_a = filedialog.askopenfilename(filetypes=[("PNG files", "*.png")])

        # Carregar a imagem e exibi-la no canvas
        self.imagem_a = Image.open(caminho_imagem_a)
        self.imagem_a = self.imagem_a.convert("RGBA")  # Garantir que a imagem tenha canal alfa
        imagem_a_tk = ImageTk.PhotoImage(self.imagem_a)
        self.canvas.create_image(0, 0, anchor="nw", image=imagem_a_tk)
        self.canvas.imagem_a_tk = imagem_a_tk  # Salvar uma referência para evitar que o garbage collector remova a imagem

    def carregar_imagem_b(self):
        # Abrir o diálogo de seleção de arquivo
        caminho_imagem_b = filedialog.askopenfilename(filetypes=[("PNG files", "*.png")])

        # Carregar a imagem e exibi-la no canvas
        self.imagem_b = Image.open(caminho_imagem_b)
        self.imagem_b = self.imagem_b.convert("RGBA")  # Garantir que a imagem tenha canal alfa
        imagem_b_tk = ImageTk.PhotoImage(self.imagem_b)
        self.canvas.create_image(0, 0, anchor="nw", image=imagem_b_tk)
        self.canvas.imagem_b_tk = imagem_b_tk  # Salvar uma referência para evitar que o garbage collector remova a imagem

    def sobrepor_imagens(self):
        if self.imagem_a is not None and self.imagem_b is not None:
            # Sobrepor a imagem A na posição desejada na imagem B
            imagem_a_copiada = self.imagem_a.copy()
            self.imagem_b.paste(imagem_a_copiada, (self.posicao_x, self.posicao_y), imagem_a_copiada)

            # Exibir a imagem resultante no canvas
            imagem_resultante_tk = ImageTk.PhotoImage(self.imagem_b)
            self.canvas.create_image(0, 0, anchor="nw", image=imagem_resultante_tk)
            self.canvas.imagem_resultante_tk = imagem_resultante_tk  # Salvar uma referência para evitar que o garbage collector remova a imagem

            # Salvar a imagem resultante (opcional)
            self.imagem_b.save("imagem_resultante.png")

if __name__ == "__main__":
    root = Tk()
    app = SobreposicaoImagensApp(root)
    root.mainloop()


KeyboardInterrupt: 

In [33]:
from tkinter import Tk, Canvas, PhotoImage, Button, filedialog
from PIL import Image, ImageTk

class SobreposicaoImagensApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Sobreposição de Imagens")

        # Variáveis para armazenar as imagens
        self.imagem_a = None
        self.imagem_b = None

        # Variáveis para armazenar as posições x e y
        self.posicao_x = 0
        self.posicao_y = 0

        # Canvas para exibir as imagens
        self.canvas = Canvas(root)
        self.canvas.pack()

        # Botões para carregar imagens e realizar a sobreposição
        self.botao_imagem_a = Button(root, text="Carregar Imagem A", command=self.carregar_imagem_a)
        self.botao_imagem_a.pack()

        self.botao_imagem_b = Button(root, text="Carregar Fundo", command=self.carregar_imagem_b)
        self.botao_imagem_b.pack()

        self.botao_sobrepor = Button(root, text="Sobrepor Imagens", command=self.sobrepor_imagens)
        self.botao_sobrepor.pack()

        # Adicionar evento de clique no Canvas
        self.canvas.bind("<Button-1>", self.mudar_posicao)

    def carregar_imagem_a(self):
        # Abrir o diálogo de seleção de arquivo
        caminho_imagem_a = filedialog.askopenfilename(filetypes=[("PNG files", "*.png")])

        # Carregar a imagem e exibi-la no canvas
        self.imagem_a = Image.open(caminho_imagem_a)
        self.imagem_a = self.imagem_a.convert("RGBA")  # Garantir que a imagem tenha canal alfa
        imagem_a_tk = ImageTk.PhotoImage(self.imagem_a)
        self.canvas.create_image(0, 0, anchor="nw", image=imagem_a_tk)
        self.canvas.imagem_a_tk = imagem_a_tk  # Salvar uma referência para evitar que o garbage collector remova a imagem

    def carregar_imagem_b(self):
        # Abrir o diálogo de seleção de arquivo
        caminho_imagem_b = filedialog.askopenfilename(filetypes=[("PNG files", "*.png")])

        # Carregar a imagem e exibi-la no canvas
        self.imagem_b = Image.open(caminho_imagem_b)
        self.imagem_b = self.imagem_b.convert("RGBA")  # Garantir que a imagem tenha canal alfa
        imagem_b_tk = ImageTk.PhotoImage(self.imagem_b)
        self.canvas.create_image(0, 0, anchor="nw", image=imagem_b_tk)
        self.canvas.imagem_b_tk = imagem_b_tk  # Salvar uma referência para evitar que o garbage collector remova a imagem

    def sobrepor_imagens(self):
        if self.imagem_a is not None and self.imagem_b is not None:
            # Sobrepor a imagem A na posição desejada na imagem B
            imagem_a_copiada = self.imagem_a.copy()
            self.imagem_b.paste(imagem_a_copiada, (self.posicao_x, self.posicao_y), imagem_a_copiada)

            # Exibir a imagem resultante no canvas
            imagem_resultante_tk = ImageTk.PhotoImage(self.imagem_b)
            self.canvas.create_image(0, 0, anchor="nw", image=imagem_resultante_tk)
            self.canvas.imagem_resultante_tk = imagem_resultante_tk  # Salvar uma referência para evitar que o garbage collector remova a imagem

            # Salvar a imagem resultante (opcional)
            self.imagem_b.save("imagem_resultante.png")

    def mudar_posicao(self, event):
        # Atualizar as posições x e y com as coordenadas do clique do mouse
        self.posicao_x = event.x
        self.posicao_y = event.y
        print(f"Posição x: {self.posicao_x}, Posição y: {self.posicao_y}")

if __name__ == "__main__":
    root = Tk()
    app = SobreposicaoImagensApp(root)
    root.mainloop()


Posição x: 17, Posição y: 14
Posição x: 297, Posição y: 224
Posição x: 364, Posição y: 264


KeyboardInterrupt: 

: 

In [1]:
from tkinter import Tk, Canvas, PhotoImage, Button, filedialog
from PIL import Image, ImageTk

class SobreposicaoImagensApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Sobreposição de Imagens")

        # Variáveis para armazenar as imagens
        self.imagem_a = None
        self.imagem_b = None
        self.imagem_resultante_tk = None

        # Variáveis para armazenar as posições x e y
        self.posicao_x = 0
        self.posicao_y = 0

        # Canvas para exibir as imagens
        self.canvas = Canvas(root)
        self.canvas.pack()

        # Botões para carregar imagens e realizar a sobreposição
        self.botao_imagem_a = Button(root, text="Carregar Imagem A", command=self.carregar_imagem_a)
        self.botao_imagem_a.pack()

        self.botao_imagem_b = Button(root, text="Carregar Imagem B", command=self.carregar_imagem_b)
        self.botao_imagem_b.pack()

        self.botao_sobrepor = Button(root, text="Sobrepor Imagens", command=self.sobrepor_imagens)
        self.botao_sobrepor.pack()

        # Adicionar evento de clique no Canvas
        self.canvas.bind("<Button-1>", self.mudar_posicao)

    def carregar_imagem_a(self):
        # Abrir o diálogo de seleção de arquivo
        caminho_imagem_a = filedialog.askopenfilename(filetypes=[("PNG files", "*.png")])

        # Carregar a imagem e exibi-la no canvas
        self.imagem_a = Image.open(caminho_imagem_a)
        self.imagem_a = self.imagem_a.convert("RGBA")  # Garantir que a imagem tenha canal alfa
        imagem_a_tk = ImageTk.PhotoImage(self.imagem_a)

        # Exibir a imagem A no canvas
        self.canvas.create_image(0, 0, anchor="nw", image=imagem_a_tk)
        self.canvas.imagem_a_tk = imagem_a_tk  # Salvar uma referência para evitar que o garbage collector remova a imagem

    def carregar_imagem_b(self):
        # Abrir o diálogo de seleção de arquivo
        caminho_imagem_b = filedialog.askopenfilename(filetypes=[("PNG files", "*.png")])

        # Carregar a imagem e exibi-la no canvas
        self.imagem_b = Image.open(caminho_imagem_b)
        self.imagem_b = self.imagem_b.convert("RGBA")  # Garantir que a imagem tenha canal alfa
        imagem_b_tk = ImageTk.PhotoImage(self.imagem_b)

        # Exibir a imagem B no canvas (imagem de fundo)
        self.canvas.create_image(0, 0, anchor="nw", image=imagem_b_tk)
        self.canvas.imagem_b_tk = imagem_b_tk  # Salvar uma referência para evitar que o garbage collector remova a imagem

    def sobrepor_imagens(self):
        if self.imagem_a is not None and self.imagem_b is not None:
            # Sobrepor a imagem A na posição desejada na imagem B (imagem de fundo)
            imagem_a_copiada = self.imagem_a.copy()
            self.imagem_b.paste(imagem_a_copiada, (self.posicao_x, self.posicao_y), imagem_a_copiada)

            # Atualizar a imagem de fundo no canvas
            imagem_resultante_tk = ImageTk.PhotoImage(self.imagem_b)
            self.canvas.create_image(0, 0, anchor="nw", image=imagem_resultante_tk)

            # Atualizar a referência para evitar que o garbage collector remova a imagem
            self.canvas.imagem_resultante_tk = imagem_resultante_tk

    def mudar_posicao(self, event):
        # Atualizar as posições x e y com as coordenadas do clique do mouse
        self.posicao_x = event.x
        self.posicao_y = event.y
        print(f"Posição x: {self.posicao_x}, Posição y: {self.posicao_y}")

if __name__ == "__main__":
    root = Tk()
    app = SobreposicaoImagensApp(root)
    root.mainloop()


Posição x: 189, Posição y: 141


KeyboardInterrupt: 

In [7]:
from tkinter import Tk, Canvas, Button, filedialog
from PIL import Image, ImageTk

class SobreposicaoImagensApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Sobreposição de Imagens")

        # Variáveis para armazenar as imagens
        self.imagem_a = None
        self.imagem_b = None

        # Variáveis para armazenar as posições x e y
        self.posicao_x = 0
        self.posicao_y = 0

        # Variável para armazenar a referência PhotoImage da imagem resultante
        self.imagem_resultante_tk = None

        # Canvas para exibir as imagens
        self.canvas = Canvas(root)
        self.canvas.pack()

        # Botões para carregar imagens e realizar a sobreposição
        self.botao_imagem_a = Button(root, text="Carregar Imagem A", command=self.carregar_imagem_a)
        self.botao_imagem_a.pack()

        self.botao_imagem_b = Button(root, text="Carregar Imagem B", command=self.carregar_imagem_b)
        self.botao_imagem_b.pack()

        self.botao_sobrepor = Button(root, text="Sobrepor Imagens", command=self.sobrepor_imagens)
        self.botao_sobrepor.pack()

    def carregar_imagem_a(self):
        # Abrir o diálogo de seleção de arquivo para imagem A
        caminho_imagem_a = filedialog.askopenfilename(filetypes=[("PNG files", "*.png")])

        # Carregar a imagem A e exibi-la no canvas
        self.imagem_a = Image.open(caminho_imagem_a)
        self.imagem_a = self.imagem_a.convert("RGBA")
        imagem_a_tk = ImageTk.PhotoImage(self.imagem_a)
        self.canvas.create_image(0, 0, anchor="nw", image=imagem_a_tk)
        self.canvas.imagem_a_tk = imagem_a_tk  # Salvar uma referência para evitar que o garbage collector remova a imagem

    def carregar_imagem_b(self):
        # Abrir o diálogo de seleção de arquivo para imagem B
        caminho_imagem_b = filedialog.askopenfilename(filetypes=[("PNG files", "*.png")])

        # Carregar a imagem B e exibi-la no canvas
        self.imagem_b = Image.open(caminho_imagem_b)
        self.imagem_b = self.imagem_b.convert("RGBA")
        imagem_b_tk = ImageTk.PhotoImage(self.imagem_b)
        self.canvas.create_image(0, 0, anchor="nw", image=imagem_b_tk)
        self.canvas.imagem_b_tk = imagem_b_tk  # Salvar uma referência para evitar que o garbage collector remova a imagem

    def sobrepor_imagens(self):
        if self.imagem_a is not None and self.imagem_b is not None:
            # Calcular as coordenadas para centralizar a imagem A na imagem B
            largura_b, altura_b = self.imagem_b.size
            largura_a, altura_a = self.imagem_a.size

            x = (largura_b - largura_a) // 2
            y = (altura_b - altura_a) // 2

            # Sobrepor a imagem A na posição calculada na imagem B
            imagem_a_copiada = self.imagem_a.copy()
            self.imagem_b.paste(imagem_a_copiada, (x, y), imagem_a_copiada)

            # Exibir a imagem resultante no canvas
            if self.imagem_resultante_tk:
                self.canvas.delete(self.imagem_resultante_tk)

            imagem_resultante_tk = ImageTk.PhotoImage(self.imagem_b)
            self.canvas.create_image(0, 0, anchor="nw", image=imagem_resultante_tk)
            self.imagem_resultante_tk = imagem_resultante_tk  # Salvar uma referência para evitar que o garbage collector remova a imagem

            # Salvar a imagem resultante (opcional)
            self.imagem_b.save("imagem_resultante.png")

if __name__ == "__main__":
    root = Tk()
    app = SobreposicaoImagensApp(root)
    root.mainloop()
